In [1]:
from citipy import citipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from config1 import weatherkey
import requests
import time
from datetime import datetime

In [29]:
#loaded a csv of the city_data_df in case kernel restarts 
file_to_load = 'Resources/city_data.csv'
city_data_df = pd.read_csv(file_to_load)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,61.05,67,58,17.40,AU,2021-12-05 23:02:11
1,Kavaratti,10.5669,72.6420,80.69,72,60,12.48,IN,2021-12-05 23:04:01
2,Jieshi,22.8134,115.8257,57.94,61,23,8.63,CN,2021-12-05 23:04:45
3,Malanje,-9.5402,16.3410,64.20,96,100,3.11,AO,2021-12-05 23:04:45
4,Mataura,-46.1927,168.8643,54.48,47,92,5.06,NZ,2021-12-05 23:04:45
...,...,...,...,...,...,...,...,...,...
555,Orotukan,62.2667,151.7000,-36.38,97,19,5.28,RU,2021-12-05 23:19:20
556,Nongstoin,25.5167,91.2667,57.38,89,100,2.28,IN,2021-12-05 23:19:20
557,Corumba,-19.0092,-57.6533,72.46,99,100,4.56,BR,2021-12-05 23:19:20
558,Mount Isa,-20.7333,139.5000,78.57,57,100,16.11,AU,2021-12-05 23:19:21


In [2]:
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
x = [25.12903645, 25.92017388, 26.62509167, -59.98969384, 37.30571269]
y = [-67.59741259, 11.09532135, 74.84233102, -76.89176677, -61.13376282]
coordinates = zip(x, y)

In [4]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

25.12903645 -67.59741259
25.92017388 11.09532135
26.62509167 74.84233102
-59.98969384 -76.89176677
37.30571269 -61.13376282


In [5]:
coordinates = list(lat_lngs)

In [6]:
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
len(cities)

612

In [7]:

city = 'Boston'
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weatherkey
city_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weatherkey + "&q=" + city
boston_data = requests.get(city_url).json()
boston_data

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 36.66,
  'feels_like': 34.23,
  'temp_min': 30.52,
  'temp_max': 41.02,
  'pressure': 1028,
  'humidity': 55},
 'visibility': 10000,
 'wind': {'speed': 3.29, 'deg': 154, 'gust': 4.36},
 'clouds': {'all': 93},
 'dt': 1638745201,
 'sys': {'type': 2,
  'id': 2001997,
  'country': 'US',
  'sunrise': 1638705498,
  'sunset': 1638738720},
 'timezone': -18000,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [8]:
lat = boston_data['coord']['lat']
lng = boston_data['coord']['lon']
max_temp = boston_data['main']['temp_max']
humidity = boston_data['main']['humidity']
clouds = boston_data['clouds']['all']
wind = boston_data['wind']['speed']
print(lat,lng,max_temp,humidity,clouds,wind)

42.3584 -71.0598 41.02 55 93 3.29


In [9]:
date = boston_data['dt']
datetime.utcfromtimestamp(date)

datetime.datetime(2021, 12, 5, 23, 0, 1)

In [10]:
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2021-12-05 23:00:01'

In [11]:
city_data = []
print("Beginning Data Retrieval     \n-----------------------------")
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [13]:
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | busselton
Processing Record 2 of Set 1 | kavaratti
Processing Record 3 of Set 1 | jieshi
Processing Record 4 of Set 1 | malanje
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | baykit
Processing Record 7 of Set 1 | plastun
Processing Record 8 of Set 1 | padang
Processing Record 9 of Set 1 | kodiak
Processing Record 10 of Set 1 | talcahuano
Processing Record 11 of Set 1 | leningradskiy
Processing Record 12 of Set 1 | sitka
Processing Record 13 of Set 1 | siocon
Processing Record 14 of Set 1 | ushuaia
Processing Record 15 of Set 1 | jijiga
Processing Record 16 of Set 1 | bathsheba
Processing Record 17 of Set 1 | kavieng
Processing Record 18 of Set 1 | isangel
Processing Record 19 of Set 1 | praia
Processing Record 20 of Set 1 | makakilo city
Processing Record 21 of Set 1 | bluff
Processing Record 22 of Set 1 | albany
Processing Record 23 of Set 1 | tikhvin
Processing Record 24 of Set 1 | salalah
Processing Record 25 of Set 1 | puerto ay

Processing Record 42 of Set 4 | georgetown
Processing Record 43 of Set 4 | alekseyevsk
Processing Record 44 of Set 4 | wonthaggi
Processing Record 45 of Set 4 | chakrata
Processing Record 46 of Set 4 | qaanaaq
Processing Record 47 of Set 4 | masumbwe
Processing Record 48 of Set 4 | aspindza
Processing Record 49 of Set 4 | lolua
City not found. Skipping...
Processing Record 50 of Set 4 | copacabana
Processing Record 1 of Set 5 | buala
Processing Record 2 of Set 5 | havoysund
Processing Record 3 of Set 5 | santa rosa
Processing Record 4 of Set 5 | kshenskiy
Processing Record 5 of Set 5 | novobirilyussy
Processing Record 6 of Set 5 | saleaula
City not found. Skipping...
Processing Record 7 of Set 5 | grand river south east
City not found. Skipping...
Processing Record 8 of Set 5 | barra do garcas
Processing Record 9 of Set 5 | tezu
Processing Record 10 of Set 5 | boa vista
Processing Record 11 of Set 5 | ribeira grande
Processing Record 12 of Set 5 | longmont
Processing Record 13 of Set 5

Processing Record 24 of Set 8 | sungai padi
Processing Record 25 of Set 8 | provost
Processing Record 26 of Set 8 | mabama
Processing Record 27 of Set 8 | qaqortoq
Processing Record 28 of Set 8 | ostrovnoy
Processing Record 29 of Set 8 | buariki
City not found. Skipping...
Processing Record 30 of Set 8 | mackay
Processing Record 31 of Set 8 | tynda
Processing Record 32 of Set 8 | kamenskoye
City not found. Skipping...
Processing Record 33 of Set 8 | sayyan
Processing Record 34 of Set 8 | maceio
Processing Record 35 of Set 8 | saint-joseph
Processing Record 36 of Set 8 | ambodifototra
City not found. Skipping...
Processing Record 37 of Set 8 | alofi
Processing Record 38 of Set 8 | sumbawa
City not found. Skipping...
Processing Record 39 of Set 8 | severo-kurilsk
Processing Record 40 of Set 8 | labuan
Processing Record 41 of Set 8 | krutinka
Processing Record 42 of Set 8 | doctor pedro p. pena
City not found. Skipping...
Processing Record 43 of Set 8 | pokhvistnevo
Processing Record 44 o

Processing Record 14 of Set 12 | benguela
Processing Record 15 of Set 12 | belmonte
Processing Record 16 of Set 12 | jaunjelgava
Processing Record 17 of Set 12 | zhezkazgan
Processing Record 18 of Set 12 | muros
Processing Record 19 of Set 12 | cap-aux-meules
Processing Record 20 of Set 12 | saint-francois
Processing Record 21 of Set 12 | nizhnyaya tavda
Processing Record 22 of Set 12 | dhuburi
Processing Record 23 of Set 12 | manadhoo
Processing Record 24 of Set 12 | daxian
City not found. Skipping...
Processing Record 25 of Set 12 | walvis bay
Processing Record 26 of Set 12 | de-kastri
Processing Record 27 of Set 12 | tual
Processing Record 28 of Set 12 | ahipara
Processing Record 29 of Set 12 | rozkishne
Processing Record 30 of Set 12 | mumbwa
Processing Record 31 of Set 12 | hervey bay
Processing Record 32 of Set 12 | vanavara
Processing Record 33 of Set 12 | sinnamary
Processing Record 34 of Set 12 | kumluca
Processing Record 35 of Set 12 | general roca
Processing Record 36 of Set

In [33]:
len(city_data)

560

In [34]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,61.05,67,58,17.40,AU,2021-12-05 23:02:11
1,Kavaratti,10.5669,72.6420,80.69,72,60,12.48,IN,2021-12-05 23:04:01
2,Jieshi,22.8134,115.8257,57.94,61,23,8.63,CN,2021-12-05 23:04:45
3,Malanje,-9.5402,16.3410,64.20,96,100,3.11,AO,2021-12-05 23:04:45
4,Mataura,-46.1927,168.8643,54.48,47,92,5.06,NZ,2021-12-05 23:04:45
...,...,...,...,...,...,...,...,...,...
555,Orotukan,62.2667,151.7000,-36.38,97,19,5.28,RU,2021-12-05 23:19:20
556,Nongstoin,25.5167,91.2667,57.38,89,100,2.28,IN,2021-12-05 23:19:20
557,Corumba,-19.0092,-57.6533,72.46,99,100,4.56,BR,2021-12-05 23:19:20
558,Mount Isa,-20.7333,139.5000,78.57,57,100,16.11,AU,2021-12-05 23:19:21


In [35]:
new_column_order = ['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed']
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Busselton,AU,2021-12-05 23:02:11,-33.6500,115.3333,61.05,67,58,17.40
1,Kavaratti,IN,2021-12-05 23:04:01,10.5669,72.6420,80.69,72,60,12.48
2,Jieshi,CN,2021-12-05 23:04:45,22.8134,115.8257,57.94,61,23,8.63
3,Malanje,AO,2021-12-05 23:04:45,-9.5402,16.3410,64.20,96,100,3.11
4,Mataura,NZ,2021-12-05 23:04:45,-46.1927,168.8643,54.48,47,92,5.06


In [36]:
city_data_df.to_csv('Resources/city_data.csv',index_label='City_ID')